In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import pandas as pd
#from ordinaloss.utils.datasets import ImageFolder, create_transform_pipeline
from ordinaloss.utils.pretrained_models import (classification_model_densenet, 
                                                classification_model_resnet, 
                                                classification_model_vgg, 
                                                DummyModel
                                                )


from ordinaloss.utils.data_utils import data_load

from ordinaloss.utils.loss_utils import SinimLoss, GirlsLoss, CSCELoss
from ordinaloss.utils.loss_utils import create_ordinal_cost_matrix
from ordinaloss.utils.loss_utils import PredictionLoss, CombinedLoss

from ordinaloss.nextgen_engine import model_engine
from ordinaloss.utils.callback_utils import PrintingCallback

import matplotlib.pyplot as plt

from torch.utils.data import DataLoader
from torch.optim import Adam, SGD
import torch

device = "cuda:0" if torch.cuda.is_available() else "cpu"
device

loaders, _, _ = data_load("../datasets/kneeKL224/", 16)


/home/dsi/itaym/anaconda3/envs/maruloss/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


loaded ordinaloss.utils.pretrained_models
loaded ordinaloss.utils.data_utils
loaded ordinaloss.utils.loss_utils
loaded ordinaloss.utils.metric_utils!!
loaded ordinaloss.nextgen_engine.model_engine


In [4]:
constraints = torch.tensor([0.2, 1, 1, 1, 1])
test_dist   = torch.tensor([0.8, 0, 0, 0, 0.1])
current_lambdas = torch.ones(5)*0.2

tensor([0.2000, 0.2000, 0.2000, 0.2000, 0.2000])

In [ ]:
model = DummyModel()

cost_matrix = create_ordinal_cost_matrix(5, cost_distance=3, diagonal_value=20)
csce_loss = CSCELoss(cost_matrix)

init_lambdas = torch.tensor([0.2, 0.2, 0.2, 0.2, 0.2], device = device)
prediction_loss = PredictionLoss(lambdas = init_lambdas)

loss_fn = CombinedLoss(csce_loss, prediction_loss)

engine = model_engine.OrdinalEngine(
    model=model, loss_fn=loss_fn, 
    device=device, loaders=loaders, n_classes=5,
    use_lr_scheduler=False, callbacks=[],
    optimizer_fn=SGD, lr=1.0e-3, weight_decay=5.0e-2)


In [ ]:
#model = classification_model_vgg("vgg16", num_classes = 5)

cost_matrix = create_ordinal_cost_matrix(5, cost_distance=3, diagonal_value=20)



#callbacks = [PrintingCallback()]
callbacks = []

engine = model_engine.OrdinalEngine(
    model=model, loss_fn=CSCELoss(cost_matrix), 
    device=device, loaders=loaders, n_classes=5,
    use_lr_scheduler=False, callbacks=callbacks,
    optimizer_fn=SGD, lr=1.0e-3, weight_decay=5.0e-2)

loaded ordinaloss.nextgen_engine.model_engine


In [127]:
cost_matrix = create_ordinal_cost_matrix(5, cost_distance=3, diagonal_value=20)


tensor([1., 1., 1., 1., 1.])

In [133]:
test_

tensor([0.2000, 0.2000, 0.2000, 0.2000, 0.2000])

In [137]:
def modify_loss_function(constraints, test_dist, current_lambdas):
    #constraints -> [0.2, 1, 1, 1, 1]
    #test_dist ->   [0.3, 0, 0, 0, 0.7]

    c = len(test_dist)
    ratios = test_dist / constraints
    violations = (ratios>1).to(torch.float32)
    ratios = [ratios[i] for i in range(c) if violations[i]==0 else 1]
    ratios = torch.tensor(ratios, dtype=torch.float32)
    
    new_lambdas = current_lambdas *ratios
    new_lambdas = new_lambdas/new_lambdas.sum()


    return new_lambdas

constraints = torch.tensor([0.2, 1, 1, 1, 1])
test_dist   = torch.tensor([0.8, 0, 0, 0, 0.1])
current_lambdas = torch.ones(5)*0.2


modify_loss_function(constraints, test_dist, current_lambdas)

tensor([0.5000, 0.1250, 0.1250, 0.1250, 0.1250])